# 14. Matrix Decompositions

Last time, I had a quastion on the matrix decompositions, and why are they used. Let's dive deeper into this. By the way, it is not only used in linear models; matrix decompositions will arrive frequntly in recommender systems, low-rank matrix approximations and dimensionality reducrtion techniques.

## General concept of matrix factorization

You frequently want to:
- Solve linear systems $Ax = f$
- Compute eigenvalues / eigenvectors
- Compute singular values / singular vectors
- Compute inverses, even sometimes determinants 
- Compute **matrix functions** like $\exp(A), \cos(A)$ (these are not elementwise functions)

In order to do this, we represent the matrix as a sum and/or product of matrices with **simpler structure**,   such that we can solve mentioned tasks faster / in a more stable form.

In [108]:
# (here goes a story using the whiteboard)
%load_ext jupyter_black

# LU

In [113]:
import numpy as np

a = np.array([[1.0, 1.0], [0.5, 1]])
a0 = a.copy()

n = a.shape[0]

In [114]:
L = np.zeros_like(a)
U = np.zeros_like(a)

for k in range(n):  # eliminating row k
    L[k, k] = 1.0
    for i in range(k + 1, n):  #
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k + 1, n):
            a[i, j] += -L[i, k] * a[k, j]
    for i in range(k, n):
        U[k, i] = a[k, i]

In [115]:
L

array([[1. , 0. ],
       [0.5, 1. ]])

In [116]:
U

array([[1. , 1. ],
       [0. , 0.5]])

In [117]:
L @ U

array([[1. , 1. ],
       [0.5, 1. ]])

In [107]:
import scipy
scipy.linalg.lu(a0)

(array([[1., 0.],
        [0., 1.]]),
 array([[1. , 0. ],
        [0.5, 1. ]]),
 array([[1. , 1. ],
        [0. , 0.5]]))

In [124]:
eps = 1.8e-17
a = np.array([[eps, 1], [1, 1.0]])
L = np.zeros_like(a)
U = np.zeros_like(a)

for k in range(n):  # eliminating row k
    L[k, k] = 1.0
    for i in range(k + 1, n):  #
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k + 1, n):
            a[i, j] += -L[i, k] * a[k, j]
    for i in range(k, n):
        U[k, i] = a[k, i]

In [127]:
L

array([[1.00000000e+00, 0.00000000e+00],
       [5.55555556e+16, 1.00000000e+00]])

In [128]:
U

array([[ 1.80000000e-17,  1.00000000e+00],
       [ 0.00000000e+00, -5.55555556e+16]])

In [126]:
print(L @ U)

[[1.8e-17 1.0e+00]
 [1.0e+00 0.0e+00]]


# Cholesky


In [180]:
a = np.array([[eps, 1], [1, 1.0]])
L = np.zeros_like(a)
a

array([[1.8e-17, 1.0e+00],
       [1.0e+00, 1.0e+00]])

In [186]:
### THIS DOES NOT WORK


for i in range(n):
    for k in range(i + 1):
        s = 0.0
        for j in range(k):
            s += L[i, j] * L[k, j]
        if i == k:
            L[i, i] = np.sqrt(a[i, i] - s)
        else:
            L[i, k] = 1.0 / L[k, k] * (a[i, k] - s)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [175]:
L

array([[4.24264069e-09, 0.00000000e+00],
       [2.35702260e+08,            nan]])

In [176]:
L @ L.T

array([[1.8e-17,     nan],
       [    nan,     nan]])

# QR 

Lets find it from Cholesky decomposition!

In [183]:
n = 10
r = 6
a = [[1.0 / (i + j + 0.5) for i in range(r)] for j in range(n)]  # Hilbert matrix
a = np.array(a)
print(a)

[[2.         0.66666667 0.4        0.28571429 0.22222222 0.18181818]
 [0.66666667 0.4        0.28571429 0.22222222 0.18181818 0.15384615]
 [0.4        0.28571429 0.22222222 0.18181818 0.15384615 0.13333333]
 [0.28571429 0.22222222 0.18181818 0.15384615 0.13333333 0.11764706]
 [0.22222222 0.18181818 0.15384615 0.13333333 0.11764706 0.10526316]
 [0.18181818 0.15384615 0.13333333 0.11764706 0.10526316 0.0952381 ]
 [0.15384615 0.13333333 0.11764706 0.10526316 0.0952381  0.08695652]
 [0.13333333 0.11764706 0.10526316 0.0952381  0.08695652 0.08      ]
 [0.11764706 0.10526316 0.0952381  0.08695652 0.08       0.07407407]
 [0.10526316 0.0952381  0.08695652 0.08       0.07407407 0.06896552]]


In [184]:
q, Rmat = np.linalg.qr(a)
e = np.eye(r)
print("Built-in QR orth", np.linalg.norm(q.T @ q - e))

Built-in QR orth 8.933073361428516e-16


In [185]:
gram_matrix = a.T.dot(a)
Rmat1 = np.linalg.cholesky(gram_matrix)
q1 = np.linalg.solve(Rmat1, a.T).T
print("Via Cholesky:", np.linalg.norm(q1.T @ q1 - e))

Via Cholesky: 1.655777363169412e-05


# LAPACK

All of this was cool, but what **actually** is used in linear regression?
* [LinearRegression](https://github.com/scikit-learn/scikit-learn/blob/6e9039160/sklearn/linear_model/_base.py#L465) fit method:
```python
        if self.positive:
            self.coef_ = optimize.nnls(X, y)[0]
            ...
        elif sp.issparse(X):
            ...
            self.coef_ = lsqr(X_centered, y)[0]
            ...
        else:
            self.coef_, _, self.rank_, self.singular_ = linalg.lstsq(X, y)
            self.coef_ = self.coef_.T
```
NNLS was your in bonus hw, so lets discover what `lsqr` and `lstsq` are.

* [scipy.linalg.lstsq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html#lstsq)
```python
    lapack_func, lapack_lwork = get_lapack_funcs((driver,
                                                 '%s_lwork' % driver),
                                                 (a1, b1))
    if driver in ('gelss', 'gelsd'):
        ...                    
        if driver == 'gelss':
            lwork = _compute_lwork(lapack_lwork, m, n, nrhs, cond)
            v, x, s, rank, work, info = lapack_func(a1, b1, cond, lwork,
                                                    overwrite_a=overwrite_a,
                                                    overwrite_b=overwrite_b)
    
        elif driver == 'gelsd':
            ...
            lwork, iwork = _compute_lwork(lapack_lwork, m, n, nrhs, cond)
             x, s, rank, info = lapack_func(a1, b1, lwork,
                                            iwork, cond, False, False)
            
    return x, resids, rank, s
    ...
    elif driver == 'gelsy':
        lwork = _compute_lwork(lapack_lwork, m, n, nrhs, cond)
        jptv = np.zeros((a1.shape[1], 1), dtype=np.int32)
        v, x, j, rank, info = lapack_func(a1, b1, jptv, cond,
                                          lwork, False, False)
        return x, np.array([], x.dtype), rank, None
```

[LAPACK](https://en.wikipedia.org/wiki/LAPACK) ("Linear Algebra Package") is a standard software library for numerical linear algebra. It provides routines for solving systems of linear equations and linear least squares, eigenvalue problems, and singular value decomposition. It also includes routines to implement the associated matrix factorizations such as LU, QR, Cholesky and Schur decomposition.

The procedures in LAPACK have a standartized names:

A LAPACK subroutine name is in the form `pmmaaa`, where:

- `p` is a one-letter code denoting the type of numerical constants used. `S`, `D` `C` and `Z`.
- `mm` is a two-letter code denoting the kind of matrix expected by the algorithm. When the code `GE` (General) is given, the subroutine expects an $n×n$ array containing the entries of the matrix. 
- `aaa` is a one- to three-letter code describing the actual algorithm implemented in the subroutine.


Finally, go to the lapack website:

### `gelss`
Computes the minimum norm solution to a real linear least squares problem using the singular value decomposition (SVD).

### `gelsd`
The problem is solved in three steps:

- (1) Reduce the coefficient matrix A to bidiagonal form with
     Householder transformations, reducing the original problem
     into a "bidiagonal least squares problem" (BLS)
- (2) Solve the BLS using a divide and conquer approach.
- (3) Apply back all the Householder transformations to solve
     the original least squares problem.

### `gelsy`
The routine first computes a QR factorization with column pivoting:
```
     A * P = Q * [ R11 R12 ]
                 [  0  R22 ]
```
 with R11 defined as the largest leading submatrix whose estimated
 condition number is less than 1/RCOND.  The order of R11, RANK,
 is the effective rank of A.

 Then, R22 is considered to be negligible, and R12 is annihilated
 by orthogonal transformations from the right, arriving at the
 complete orthogonal factorization:
 ```
    A * P = Q * [ T11 0 ] * Z
                [  0  0 ]
```
 The minimum-norm solution is then
```
    X = P * Z**T [ inv(T11)*Q1**T*B ]
                 [        0         ]
```
 where Q1 consists of the first RANK columns of Q.

### Bonus task 13: gelss (easy)
Implement `gelss` (with numpy, python and scipy.svd).
### Bonus task 14: gelsd
Implement `gelsd` (with numpy and python ans scipy.svd). 
### Bonus task 15: gelsy
Implement `gelsy` (with numpy, python and scipy.qr).
### Bonus task 16: Cholesky
Fix bugs in the Cholesky decomposition algorithm.

**TODO:** [scipy.sparse.linalg.lsqr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lsqr.html)


---

This notebook was adapted from **Mathematics for Machine Learning** book by Marc Deisenroth, Aldo Faisal and Cheng Ong and Ivan Oseledec **course on numerical linear algebra**, and the **LAPACK documentation**.